In [25]:
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import time
plt.rc('font', size=12)

# Filter out deprecated warnings
import warnings
warnings.filterwarnings("ignore")

In [99]:
data = pd.read_csv('../kennedy/victoria.csv')
data = data.drop(columns='Unnamed: 0')

In [38]:
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

SUMMER = [12, 1, 2]
AUTUMN = [3, 4, 5]
WINTER = [6, 7, 8]
SPRING = [9, 10, 11]

In [115]:
vic = data[[TIME, PRICE, GENERATION, DEMAND]]

# Since the first date is at 00:00:00, the first period should be 48
period = [48]
x = 1
while x < len(vic):
    for i in range(48):
        period.append(i+1)
        x += 1

vic.insert(1, 'Period', period)

In [29]:
def first_cycle(spot_price):
    """ Returns first indexes of the first periods for the first cycles """
    
    max_price = 0
    min_price = 999999

    for i in range(48-6):
        """ Finds which 6 periods have the most sum and least sum 
            by going through 1 to 6, 2 to 7 and so on """

        curr = spot_price.iloc[i:i+6 ,0].sum()

        if curr < min_price:
            min_price = curr
            min_i = i  # Stores the first index of the max period

        if curr > max_price:

            max_price = curr
            max_i = i  # Stores the first index of the min period
            
    min_, max_ = store_index(min_i, max_i)
                
    return min_, max_

In [30]:
def sec_cycle(spot_price, min_index, max_index):
    """ Returns first indexes of the first periods for the second cycles """
    
    remaining = list(spot_price.index)
    sec_min_price = 999999
    sec_max_price = 0
    to_remove = min_index + max_index
    
    # remove all periods after max
    remaining = remaining[:remaining.index(to_remove[-1])+1]
    
    for index in to_remove:
        remaining.remove(index)

    for i in range(len(remaining) - 6):

        # make sure the next six indexes are increment of 1
        if remaining[i] == (remaining[i+5] - 5):

            curr_sum = spot_price.iloc[remaining[i]:remaining[i]+6 ,0].sum()

            if curr_sum < sec_min_price:
                sec_min_price = curr_sum
                sec_min_i = remaining[i]  # Stores the first index of the second max period

            if curr_sum > sec_max_price:

                sec_max_price = curr_sum
                sec_max_i = remaining[i]  # Stores the first index of the second min period
                
    min_, max_ = store_index(sec_min_i, sec_max_i)
    
    min_index += min_
    max_index += max_
                
    return min_index, max_index

In [31]:
def store_index(index1, index2):
    """ Store the rest of the max and min price indexes """
    list1 = []
    list2 = []
    for i in range(6):
        list1.append(index1 + i)
        list2.append(index2 + i)
        
    return list1, list2

In [152]:
def algorithm2(ori_df):
    """ Finds optimal charge and discharge period from the mean """

    summer_df = ori_df[(ori_df['month'] == 1) | (ori_df['month'] == 2) | (ori_df['month'] == 12)]
    autumn_df = ori_df[(ori_df['month'] == 3) | (ori_df['month'] == 4) | (ori_df['month'] == 5)]
    winter_df = ori_df[(ori_df['month'] == 6) | (ori_df['month'] == 7) | (ori_df['month'] == 8)]
    spring_df = ori_df[(ori_df['month'] == 9) | (ori_df['month'] == 10) | (ori_df['month'] == 11)]
    
    seasons_df = [summer_df, autumn_df, winter_df, spring_df]
    seasons_name = ['summer', ' autumn', 'winter', 'spring']
    discharge_period = []
    charge_period = []
    for i in range(len(seasons_df)):
        spot_price = seasons_df[i].groupby(['Period'])[[PRICE]].mean()

        # First cycle
        min_, max_ = first_cycle(spot_price)

        # Second cycle
        # Comment line 11 if only want one cycle
        min_, max_ = sec_cycle(spot_price, min_, max_)
        
        # the charge and discharge period are fixed in Algorithm 2, +1 to get their periods
        min_ = list(np.asarray(min_) + 1)
        max_ = list(np.asarray(max_) + 1)
        
        discharge_period.append(max_)
        charge_period.append(min_)
    
    return charge_period, discharge_period

In [123]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, 'Period', PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    df['revenue'] = 0
    df['decision'] = 0
    
    # I removed the first row because first row is 00:00:00, 
    # which is the last period from the previous day
    #df = df.drop([0], axis=0)
    
    # convert to datetime
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
    
    df['weekday'] = df[TIME].apply(lambda t: t.weekday())
    df['weekday'] = df['weekday'] + 1
    
    df['month'] = df[TIME].dt.month
    
    return df

In [150]:
def get_season(month):
    
    if month in SUMMER:
        return 0
    elif month in AUTUMN:
        return 1
    elif month in WINTER:
        return 2
    elif month in SPRING:
        return 3

In [156]:
def find_all(ori_df):
    """ Returns a completed dataframe """
    """ This is the main function, calling this function will automatically run all other functions """
    
    start = time.time()
    
    df = create_df(ori_df)
    charge_period, discharge_period = algorithm2(df)
    print(charge_period[0])
    print(discharge_period[0])
    df = df.reset_index(drop=True)
    df.index += 1
    
    for i in list(df.index):

        period = df.at[i, "Period"]
        price = df.at[i, PRICE]

        """ Find Opening Cap """
        # Starts from 1 because the first row was removed
        if i != 1:
            df.at[i,"opening"] = df.at[i-1,"closing"]

        opening_cap = math.ceil(df.at[i, "opening"])




        """ Find raw_power """
        season = get_season(df.at[i, 'month'])
        
        if period in charge_period[season]:
                df.at[i, "raw_power"] = -math.floor(min(POWER,(CAPACITY-opening_cap)/(CHARGE_EFF/100)*2))

        elif period in discharge_period[season]:
            df.at[i, "raw_power"] = math.floor(min(POWER,opening_cap/(DISCHARGE_EFF/100)*2))

        rawPower = df.at[i, "raw_power"]




        """ Find dispatch """
        if rawPower < 0:
            eff = 1

        else:
            eff = DISCHARGE_EFF / 100

        df.at[i,"dispatch"] = math.ceil(rawPower / 2 * eff)
        dispatch = df.at[i, "dispatch"]



        """ Find Closing Cap """
        if dispatch < 0:
            eff = CHARGE_EFF / 100

        else:
            eff = 100 / DISCHARGE_EFF

        df.at[i,"closing"] = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))



        """ Find revenue """
        if dispatch < 0:
            factor = 1/MLF

        else:
            factor = MLF

        df.at[i,"revenue"] = math.ceil(price * dispatch * factor)
        
    
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    end = time.time()
    print("Time Complexity for running the entire Algorithm 2: {time_taken}s".format(time_taken = end-start))
    
    return df

In [159]:
# With seasons
# problem now is that in summer, cheapest periods are all before period 24 
# and most expensive periods are all after period 24
# Hence only got one cycle in summer

alg = find_all(vic)

[4, 5, 6, 7, 8, 9, 15, 16, 17, 18, 19, 20]
[33, 34, 35, 36, 37, 38, 26, 27, 28, 29, 30, 31]
Total revenue in the dataset: 53588035
Total days in the dataset: 1322.0208333333333
Revenue per day: 40534.939880549035
Time Complexity for running the entire Algorithm 2: 4.113589286804199s


In [ ]:
summer_df = vic2[(vic2['month'] == 12) | (vic2['month'] == 1) | (vic2['month'] == 2)]
autumn_df = vic2[(vic2['month'] == 5) | (vic2['month'] == 3) | (vic2['month'] == 4)]
winter_df = vic2[(vic2['month'] == 6) | (vic2['month'] == 7) | (vic2['month'] == 8)]
spring_df = vic2[(vic2['month'] == 9) | (vic2['month'] == 10) | (vic2['month'] == 11)]

# Include other factors (seasons) into our algorithm